In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("precision",16)
%matplotlib inline

In [10]:
class LSTM_Config(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.lr = 0.0001
        self.num_layers = 1
        self.hidden_size = 100
        self.input_size = 1
        self.output_size = 1
        self.num_epochs = 1000
        self.batch_size = 50
        self.step_size = 5
        self.reuse = False
        self.print_interval = 1

In [11]:
class LSTM_Model(object):
    def __init__(self,scope,config):
        self.config = config
        with tf.variable_scope(scope):
            self._build_graph()
    
    def _build_graph(self):
        self._init_placeholders()
        self._build_model()
        self._setup_training()
        
    def _init_placeholders(self):
        self.input_ph = tf.placeholder(
            tf.float32, 
            [None, self.config.step_size, self.config.input_size],
            name="input_ph"
        )
        self.targets_ph = tf.placeholder(
            tf.float32,
            [None, self.config.output_size],
            name="targets_ph"
        )
        
    def _build_model(self):
        with tf.variable_scope("model", reuse=self.config.reuse):
            lstm_cell = tf.contrib.rnn.LSTMCell(
                self.config.hidden_size,
                activation=tf.nn.relu,
                state_is_tuple=True
            )
            
            batch_size = tf.shape(self.input_ph)[0]
            step_size = tf.shape(self.input_ph)[1]
            
            self.state_in = lstm_cell.zero_state(batch_size, tf.float32)
            
            self.model, self.state = tf.nn.dynamic_rnn(
                lstm_cell,
                self.input_ph,
                initial_state=self.state_in,
                time_major=False
            )
            
            c,h = self.state
            self.state_out = (c,h)
            self.model = tf.reshape(self.model, shape=[-1,self.config.hidden_size])
            
            weights = tf.Variable(tf.random_uniform(
                [self.config.hidden_size, self.config.output_size],
                minval=-0.05, maxval=0.05
            ))
            biases = tf.Variable(tf.random_uniform(
                [self.config.output_size],
                minval=-0.05, maxval=0.05
            ))
            self.model = tf.nn.xw_plus_b(self.model, weights, biases, name="model")
            self.model = tf.reshape(self.model, shape=[batch_size, step_size, self.config.output_size])
    
    def _setup_training(self):
        self.loss = tf.reduce_sum(tf.square(self.model - self.targets_ph))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.config.lr)
        self.training = self.optimizer.minimize(self.loss)

In [4]:
%%time
data = pd.read_csv("./train/train.csv", dtype={'acoustic_data':np.int16, 'time_to_failure':np.float32})

Wall time: 2min 4s


In [9]:
signal_train, fail_train = data['acoustic_data'].values[41:100041], \
                            data['time_to_failure'].values[41:100041]
next_signal_train = data['acoustic_data'].values[42:100042]
signal_test, fail_test = data['acoustic_data'].values[100043:120043], \
                            data['time_to_failure'].values[100043:120043]
next_signal_test = data['acoustic_data'].values[100043:120043]

print(len(signal_train))
print(len(next_signal_train))
print(len(signal_test))

100000
100000
20000


In [15]:
config =  LSTM_Config()
    
tf.reset_default_graph()

model = LSTM_Model('model', config)


print_interval = config.print_interval
init_op = tf.global_variables_initializer()

index_in_epoch = 0
perm_array = np.arange(signal_train.shape[0])
np.random.shuffle(perm_array)

def next_batch(batch_size):
    global index_in_epoch, signal_train, next_signal_train, perm_array
    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > signal_train.shape[0]:
        np.random.shuffle(perm_array)
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch
    return signal_train[perm_array[start:end]], next_signal_train[perm_array[start:end]]

with tf.Session() as sess:

    sess.run([init_op])

    train_size = signal_train.shape[0]
    lstm_state = np.zeros((2, config.batch_size, config.hidden_size))


    for iteration in range(int(config.num_epochs*train_size/config.batch_size)):
        if index_in_epoch == 0:
            lstm_state = np.zeros((2, config.batch_size, config.hidden_size))

        signal_batch, target_batch = next_batch(config.batch_size)

        training_feed = {
            model.input_ph : signal_batch,
            model.targets_ph : target_batch,
            model.state_in[0] : lstm_state[0],
            model.state_in[1] : lstm_state[1]
        }

        loss, _, lstm_state = sess.run([model.loss, model.training, model.state],
                feed_dict=training_feed)
        if iteration % int(5*train_size/config.batch_size) == 0:
            eval_feed = {
                model.input_ph : signal_train,
                model.targets_ph : next_signal_train,
                model.state_in[0] : lstm_state[0],
                model.state_in[1] : lstm_state[1]
            }
            mse_train = model.loss.eval(feed_dict=eval_feed)

            valid_feed = {
                model.input_ph : signal_test,
                model.targets_ph : next_signal_test,
                model.state_in[0] : lstm_state[0],
                model.state_in[1] : lstm_state[1]
            }
            mse_valid = model.loss.eval(feed_dict=valid_feed)
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*config.batch_size/train_size, mse_train, mse_valid))

ValueError: Cannot feed value of shape (50,) for Tensor 'model/input_ph:0', which has shape '(?, 5, 1)'